<a href="https://colab.research.google.com/github/Puurun/fastai_code_example/blob/main/12_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [ ]:
from fastbook import *

In [ ]:
from fastai.text.all import *

In [ ]:
path = untar_data(URLs.HUMAN_NUMBERS)

In [ ]:
Path.BASE_PATH = path

In [ ]:
path.ls()

NameError: ignored

In [ ]:
lines = L()
f = open(path/'train.txt')
lines += L(*f.readlines())
f = open(path/'valid.txt')
lines += L(*f.readlines())
lines

In [ ]:
lines = L()
with open(path/'train.txt') as f: lines += L(*f.readlines())
with open(path/'valid.txt') as f: lines += L(*f.readlines())
lines

In [ ]:
text = ' . '.join([l.strip() for l in lines])
text[:100]

In [ ]:
tokens = text.split(' ')
tokens[:10]

In [ ]:
vocab = L(*tokens).unique()

In [ ]:
vocab

In [ ]:
word2idx = {w:i for i, w in enumerate(vocab)}

In [ ]:
nums = L(word2idx[i] for i in tokens)

In [ ]:
nums

In [ ]:
# creating data => sequence of three words
L((tokens[i:i+3], tokens[i+3]) for i in range(0, len(tokens)-4, 3))

In [ ]:
seqs = L((tensor(nums[i:i+3]), tensor(nums[i+3])) for i in range(0, len(nums)-4, 3))

In [ ]:
bs=64
cut = int(len(seqs) * 0.8)
dls = DataLoaders.from_dsets(seqs[:cut], seqs[cut:], bs=bs, shuffle=False)

In [ ]:
first(dls.train)[0].shape

In [ ]:
class LMModel1(Module):
    def  __init__(self, vocab_sz, n_hidden):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)
        self.h_h = nn.Linear(n_hidden, n_hidden)
        self.h_o = nn.Linear(n_hidden, vocab_sz)
    
    def forward(self, x):
        h = F.relu(self.h_h(self.i_h(x[:,0])))
        h = h + self.i_h(x[:,1])
        h = F.relu(self.h_h(h))
        h = h + self.i_h(x[:,2])
        h = F.relu(self.h_h(h))
        return self.h_o(h)

In [ ]:
learn = Learner(dls, LMModel1(len(vocab), 64), loss_func=F.cross_entropy, metrics=accuracy)

In [ ]:
learn.fit_one_cycle(4, 1e-3)

In [ ]:
class LMModel2(Module):
    def __init__(self, vocab_sz, n_hidden):
        self.i_h = Embedding(vocab_sz, n_hidden)
        self.h_h = nn.Linear(n_hidden, n_hidden)
        self.h_o = nn.Linear(n_hidden, vocab_sz)
    def forward(self, x):
        h = 0
        for i in range(3):
            h = h + self.i_h(x[:,i])
            h = F.relu(self.h_h(h))
        return self.h_o(h)

In [ ]:
learn2 = Learner(dls, LMModel2(len(vocab), 64), loss_func=F.cross_entropy, metrics=accuracy)

In [ ]:
learn2.fit_one_cycle(4, 1e-3)

In [ ]:
class LMModel3(Module):
    def __init__(self, vocab_sz, n_hidden):
        self.i_h = Embedding(vocab_sz, n_hidden)
        self.h_h = nn.Linear(n_hidden, n_hidden)
        self.h_o = nn.Linear(n_hidden, vocab_sz)
        self.h = 0
    def forward(self, x):
        for i in range(3):
            self.h = self.h + self.i_h(x[:,i])
            self.h = F.relu(self.h_h(self.h))
        out = self.h_o(self.h)
        self.h = self.h.detach()
        return out
    def reset(self): self.h = 0

In [ ]:
def group_chunks(ds, bs):
    m = len(ds)//bs
    new_ds = L()
    for i in range(m):
        new_ds += L(ds[i + m*j] for j in range(bs))
    return new_ds


In [ ]:
cut = int(len(seqs) * 0.8)
dls2 = DataLoaders.from_dsets(group_chunks(seqs[:cut], bs), group_chunks(seqs[cut:], bs), bs=bs, drop_last=True, shuffle=False)

In [ ]:
learn3 = Learner(dls2, LMModel3(len(vocab), 64), loss_func=F.cross_entropy, metrics=accuracy)

In [ ]:
learn3.fit_one_cycle(10, 1e-3)

In [ ]:
sl = 16
seqs2 = L((tensor(nums[i:i+sl]), tensor(nums[i+1:i+sl+1])) for i in range(0, len(nums)-sl-1, sl))
cut2 = int(len(seqs2) * 0.8)
dls3 = DataLoaders.from_dsets(group_chunks(seqs2[:cut2], bs), group_chunks(seqs2[cut2:], bs), bs=bs, drop_last=True, shuffle=False)

In [ ]:
seqs2[0]

(tensor([0, 1, 2, 1, 3, 1, 4, 1, 5, 1, 6, 1, 7, 1, 8, 1]),
 tensor([1, 2, 1, 3, 1, 4, 1, 5, 1, 6, 1, 7, 1, 8, 1, 9]))

In [ ]:
[L(vocab[o] for o in s) for s in seqs2[0]]

[(#16) ['one','.','two','.','three','.','four','.','five','.'...],
 (#16) ['.','two','.','three','.','four','.','five','.','six'...]]

In [ ]:
first(dls3.valid)

(tensor([[29, 26,  3,  ...,  1,  8, 29],
         [ 0, 28, 18,  ..., 28, 20,  1],
         [28, 22,  6,  ...,  0, 28, 22],
         ...,
         [ 9, 29,  8,  ...,  1,  9, 29],
         [ 1,  9, 29,  ...,  3,  1,  9],
         [28, 20,  2,  ...,  9, 28, 20]]),
 tensor([[26,  3,  1,  ...,  8, 29, 26],
         [28, 18,  1,  ..., 20,  1,  8],
         [22,  6,  1,  ..., 28, 22,  8],
         ...,
         [29,  8, 28,  ...,  9, 29,  8],
         [ 9, 29,  8,  ...,  1,  9, 29],
         [20,  2,  1,  ..., 28, 20,  4]]))

In [ ]:
class LMModel4(Module):
    def __init__(self, vocab_sz, n_hidden):
        self.i_h = Embedding(vocab_sz, n_hidden)
        self.h_h = nn.Linear(n_hidden, n_hidden)
        self.h_o = nn.Linear(n_hidden, vocab_sz)
        self.h = 0
    def forward(self, x):
        out = []
        for i in range(sl):
            self.h = self.h + self.i_h(x[:,i])
            self.h = F.relu(self.h_h(self.h))
            out.append(self.h_o(self.h))
        self.h = self.h.detach()
        return torch.stack(out, dim=1)
    def reset(self): self.h = 0

In [ ]:
len(vocab)

30

In [ ]:
# CrossEntropyLossFlat()
# since our output is size(bs, sl, vocab_sz) and target size is (bs, sl), we need to flat it out to apply cross entropy.
# in this case bs=64 sl=16 vocab_sz=30
# so, inp.shape=(1024,30) tar.shape=(1024)
def cross_entropy(inp, tar):
    return F.cross_entropy(inp.view(-1, len(vocab)), tar.view(-1))

In [ ]:
learn4 = Learner(dls3, LMModel4(len(vocab), 64), loss_func=cross_entropy, metrics=accuracy, cbs=ModelResetter)

In [ ]:
learn4.fit_one_cycle(15, 2e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.083259,1.807938,0.557861,00:01
1,1.089662,1.844058,0.546712,00:01
2,1.086510,1.719083,0.556396,00:01
3,1.070494,1.733915,0.561605,00:01
4,1.044683,1.677386,0.576660,00:01
5,1.011502,1.657302,0.584310,00:01
6,0.979341,1.672398,0.583089,00:01
7,0.955955,1.689476,0.579346,00:01
8,0.920018,1.771754,0.566488,00:01
9,0.892649,1.726435,0.585286,00:01
